In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
    
import h5py
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
try:
    import dill as pickle
except ImportError:
    import pickle

# Loading Precomputed Data

In [3]:
# Loading Image Features
train_features = load(open("/home/vinit/Desktop/Projects/Image Captioning/Dataset/encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=6000


In [4]:
# Load Train Descriptions, which contains, 5 captions corresponding to a key
train_descriptions = load(open("/home/vinit/Desktop/Projects/Image Captioning/Dataset/train_descriptions.pkl", "rb"))
print('Photos: train=%d' % len(train_descriptions))

Photos: train=6000


In [5]:
# Loading embedding_matrix
embedding_matrix = load(open("/home/vinit/Desktop/Projects/Image Captioning/Dataset/Embedding_Matrix.pkl", "rb"))
print('Embedding Matrix shape = (%d, %d)' % (embedding_matrix.shape))

Embedding Matrix shape = (2057, 200)


In [6]:
# Loading wordtoix
wordtoix = load(open("/home/vinit/Desktop/Projects/Image Captioning/Dataset/wordtoix.pkl", "rb"))
print('Wordtoix Length = %d' % len(wordtoix))

Wordtoix Length = 2056


# Data Generator

In [7]:
max_length = 34 # computer from file 2
epochs = 10
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [8]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0
    return X1, X2, y

In [9]:
max_length = 35 # computer from file 2
epochs = 10
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [10]:
train_features['2513260012_03d33305cf.jpg'].shape

(2048,)

# Defining the Model

In [11]:
max_length = 35 
vocab_size = embedding_matrix.shape[0]
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath
embedding_dim = embedding_matrix.shape[1]
max_length,vocab_size ,embedding_dim

(35, 2057, 200)

In [12]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
my_model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [13]:
my_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 35, 200)      411400      input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
__________________________________________________________________________________________________
dropout_2 

In [14]:
my_model.layers[2]

In [15]:
my_model.layers[2].set_weights([embedding_matrix])
my_model.layers[2].trainable = False

In [16]:
my_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001))

In [17]:
my_model.save('./model_weights/model_' + 'test' + '.h5')

In [18]:
epochs = 40
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [19]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    my_model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    my_model.save('./model_weights/model_' + str(i) + '.h5')

Epoch 1/1
1000/1000 [==============================] - 628s 628ms/step - loss: 4.3032
Epoch 1/1
1000/1000 [==============================] - 606s 606ms/step - loss: 3.5316
Epoch 1/1
1000/1000 [==============================] - 626s 626ms/step - loss: 3.2799
Epoch 1/1
1000/1000 [==============================] - 609s 609ms/step - loss: 3.1282
Epoch 1/1
1000/1000 [==============================] - 608s 608ms/step - loss: 3.0155
Epoch 1/1
1000/1000 [==============================] - 648s 648ms/step - loss: 2.9290
Epoch 1/1
1000/1000 [==============================] - 674s 674ms/step - loss: 2.8612
Epoch 1/1
1000/1000 [==============================] - 667s 667ms/step - loss: 2.7982
Epoch 1/1
1000/1000 [==============================] - 736s 736ms/step - loss: 2.7479
Epoch 1/1
1000/1000 [==============================] - 758s 758ms/step - loss: 2.7057
Epoch 1/1
1000/1000 [==============================] - 662s 662ms/step - loss: 2.6661
Epoch 1/1
1000/1000 [==============================] -

In [22]:
with open('/home/vinit/Desktop/Projects/Image Captioning/Caption_generator_Model.pkl', 'wb') as model:
    pickle.dump(my_model, model)